In [7]:
import glob
import monai
import os
from os.path import basename, dirname, exists, isdir, join, split
from pathlib import Path
import functools
import multiprocessing as mp
import os
from functools import partial
import numpy as np
import pandas as pd
from toolz.itertoolz import groupby
import SimpleITK as sitk


root_dir='/workspaces/prost_anatomy_seg/data/10'


def group_files(root_dir):
    """ 
    based on name will group the files into triplets of related files
    """
    all_files=os.listdir(root_dir)
    grouped_by_master= groupby(lambda path : path.split('_')[0],all_files)
    return list(dict(grouped_by_master).items())

# https://github.com/jakubMitura14/forPicaiDocker/blob/main/testRadiomics/getGoldLabelsRadiomics.py

def analyze_single_label(uniq_num,centers, big_mask, connected,cover_min,in_min):
    infered=(connected==uniq_num)
    total=np.sum(infered.flatten())
    inn= np.sum(np.logical_and(infered,big_mask).flatten())/total
    cov= np.sum(np.logical_and(infered,centers).flatten())/np.sum(centers.flatten())
    res= (inn>in_min) #and (cov>cover_min)
    return res

def analyze_case(group,root_dir):
    num,paths=group
    paths= list(map(lambda p: f"{root_dir}/{p}",paths))

    centers=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'centers' in pathh  ,paths))[0])).astype(bool)
    big_mask=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'big_mask' in pathh  ,paths))[0])).astype(bool)
    inferred=sitk.ReadImage(list(filter(lambda pathh: 'inferred' in pathh  ,paths))[0])
    if(np.sum(sitk.GetArrayFromImage(inferred))==0):
        return 1.0
    connected=sitk.GetArrayFromImage(sitk.ConnectedComponent(inferred))
    uniqq=np.unique(connected)
    uniqq= list(filter(lambda el:el>0,uniqq))
    cover_min=0.0
    in_min=0.8
    res= list(map(lambda uniq_num: analyze_single_label(uniq_num,centers, big_mask, connected,cover_min,in_min), uniqq))
    res= np.mean(np.array(res).astype(int))
    return res

grouped=group_files(root_dir)
# def chunks(xs, n):
#     n = max(1, n)
#     return (xs[i:i+n] for i in range(0, len(xs), n))
# # aa=list(map(lambda g:analyze_case(g,root_dir)  ,grouped))
# grouped=list(chunks(grouped,6))
# grouped

[[('734',
   ['734_big_mask.nii.gz', '734_inferred.nii.gz', '734_centers.nii.gz']),
  ('1732',
   ['1732_centers.nii.gz', '1732_big_mask.nii.gz', '1732_inferred.nii.gz']),
  ('1803',
   ['1803_big_mask.nii.gz', '1803_centers.nii.gz', '1803_inferred.nii.gz']),
  ('724',
   ['724_inferred.nii.gz', '724_centers.nii.gz', '724_big_mask.nii.gz']),
  ('1817',
   ['1817_inferred.nii.gz', '1817_big_mask.nii.gz', '1817_centers.nii.gz']),
  ('1236',
   ['1236_inferred.nii.gz', '1236_big_mask.nii.gz', '1236_centers.nii.gz'])],
 [('1329',
   ['1329_big_mask.nii.gz', '1329_inferred.nii.gz', '1329_centers.nii.gz']),
  ('821',
   ['821_centers.nii.gz', '821_big_mask.nii.gz', '821_inferred.nii.gz']),
  ('841',
   ['841_centers.nii.gz', '841_big_mask.nii.gz', '841_inferred.nii.gz']),
  ('944',
   ['944_centers.nii.gz', '944_big_mask.nii.gz', '944_inferred.nii.gz']),
  ('1509',
   ['1509_inferred.nii.gz', '1509_centers.nii.gz', '1509_big_mask.nii.gz']),
  ('1111',
   ['1111_big_mask.nii.gz', '1111_center

In [2]:
np.mean(aa)

0.7857894736842105

In [43]:
g=list(filter(lambda el:el[0]=='5' ,grouped))[0]
analyze_case(g,root_dir)


[1, 2]


('5', 0.5)

In [48]:
np.mean(aa)


0.6862118595279216

In [3]:

def get_is_correct(curr_in,curr_twos):
    total = curr_in.sum()
    curr_percent_in=np.zeros(1)
    curr_percent_covered=np.zeros(1)

    if(total.item()>0):
        curr_percent_in=curr_in.sum()/(total)
    if(curr_twos.sum().item()>0):
        curr_percent_covered=  ((curr_in) & (curr_twos)).sum()/ curr_twos.sum()

    return (curr_percent_in>0.8 and curr_percent_covered>0.4)


def analyze_case_sum(group,root_dir):
    num,paths=group
    paths= list(map(lambda p: f"{root_dir}/{p}",paths))

    centers=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'centers' in pathh  ,paths))[0])).astype(bool)
    big_mask=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'big_mask' in pathh  ,paths))[0])).astype(bool)
    inferred=sitk.ReadImage(list(filter(lambda pathh: 'inferred' in pathh  ,paths))[0])
    connected=sitk.GetArrayFromImage(inferred)
    connected=(connected>0)
    # uniqq= list(filter(lambda el:el>0,uniqq))
    cover_min=0.5
    in_min=0.8
    
    inn = connected & big_mask

    res=get_is_correct(inn,centers)
    # res= analyze_single_label(True,centers, big_mask, connected,cover_min,in_min)
    res= np.mean(np.array(res).astype(int))
    return (num,res)

grouped=group_files(root_dir)

bb=list(map(lambda g:analyze_case_sum(g,root_dir)  ,grouped))

In [4]:
# np.mean(bb)
bbb= list(map(lambda tupl: tupl[1],bb))
# np.mean(bbb)
np.mean(bbb)

0.04421052631578947